In [8]:
import pandas as pd
import csv
import re 

In [ ]:
import pandas as pd
import re

# csv_path = '/Users/log/Downloads/fs_inst_gemini-1.5-pro-002_temp_10_longest (2).csv'
csv_path = '/Users/log/Downloads/fs_inst_llama_sambanova_405b_temp_10_longest (4).csv'
jsonl_path = '/Users/log/Github/textual_grounding/data/drop_break/test.jsonl'

responses = pd.read_csv(csv_path)
data = pd.read_json(jsonl_path, lines=True)
print(len(responses))

def extract_answer(text):
    pattern = re.compile(r"\{([-]?[0-9,]*\.?[0-9]+)\}")
    match = pattern.search(str(text))
    num_str = match.group(1) if match else "-99999999999999"
    num_str_clean = num_str.replace(',', '')  # '9759417'
    numeric_value = float(num_str_clean)      # 9759417.0 (float)
    return numeric_value

bad_questions = {
    "nfl_1184_7dfd2b64-f39e-4bb4-aeb0-1900adda6018",
    "history_2170_9b27311d-81ec-4f40-a4af-7ead916d5859",
    "nfl_16_9eb68f5c-0c59-4850-9f2d-e6bbb80cbfa0",
    "history_1167_f7cbde06-0f50-46fc-9146-aa0968af570f",
    "nfl_2151_2cf40f99-789c-4530-ade1-a3f3aff3ca6c",
    "history_1276_3cf695a7-f48c-4a59-93a6-1475962ee4c8",
    "history_254_14720a39-5dd9-498d-a922-8b77af3a4dff",
    "history_200_ac47eb17-6d08-488e-9f69-8d1e0d018767",
    "history_200_6153eb8b-88b3-40b7-9644-129f36fde149",
    "nfl_2197_a0555e2e-d0a1-4c3b-bfa9-834fef7f90c9",
    "history_241_39b1772e-28ba-44d4-be18-52f24d87bf09",
    "history_1298_65816218-01c4-4071-b10e-32018bf3555f",
    "history_1859_7c7aeed2-3f87-483a-824b-c8bd10d576f8",
    "nfl_1672_0d4f9fa3-1999-467f-b3d2-c61bf0e278dc",
    "history_1373_3994c80e-788b-4bdf-a34c-ba1a44dbca5f",
    "history_104_96d19098-478d-4c14-a33f-cd8a45966f16",
    "history_104_96590b11-eb05-4e81-99e5-58366c63d764",
    "history_2064_e3ee593d-095d-4373-83fe-6399c45feea9",
    "nfl_2234_95d3a1ab-939a-4a83-a6c1-30aeff0cbef0",
    "history_227_bc94d5da-bedc-4713-afd4-b6014391d766", # rest are gemini
    "history_2162_9e860e5c-a113-41b4-b609-b1b6223727cf",
    "nfl_2234_a929d4a2-7659-4aad-86a7-6eecf9324d8c",
    "nfl_2234_50615917-af5a-415f-bd18-41b6c4cd78e4",
    "history_1313_797e963c-52f2-4f8e-ba70-5a382cda0d85",
    "nfl_2125_92420219-58dd-4c72-87cb-8804cbb50927",
    "nfl_2125_c95ac764-33f0-423b-815f-c4d9cfd125bb",
    "nfl_1390_143a118b-f487-451d-8a10-9a58b9b6b862",
    "history_1418_da95efc1-0a9d-4232-9af5-68dc6e3e69df",
    "nfl_2034_aa94973c-862d-4601-be2d-65f8d131949d",
    "history_1665_f452e8c4-5eb3-48e9-9cfc-927c2b03ff5d",
    "history_241_9c99ae46-9e98-4e8a-ae30-09c0c96c97e9",
    "history_1898_8b859eea-bf2c-4c53-ab6d-c914a42a13c1",
    "history_1907_2e5f562d-7c22-4b82-9532-2cd8e7b68ca6",
    "history_2086_925de638-cd7e-4b29-ac5c-b72836d098e4",
    "history_1811_9b70bc55-b697-45ea-9bf0-565c444e2fa4",
    "history_1408_7b6e7625-a8ba-4cde-8496-1f6c222b2bc9",
    "nfl_1769_09b90451-e508-4b2e-9c96-948b18573bac",
    "history_2609_8325b66f-987a-443c-9940-ed95c9d80d6c",
    "history_2609_0ca05637-43b9-444f-9c0a-3d27a214d83d",
    "history_2609_eb1581c9-d297-47aa-a54d-c30be2e81530",
    "history_2676_4ae63d91-02cb-4299-809d-4d7128520a87",
    "nfl_1838_38327bd5-01a4-4342-b7e0-cb209f2b4655"
}

good_questions = {
    "history_1863_b73c649b-7d9e-433b-8ac0-c91de4b48a6c",
    "history_1042_8d1d1067-668e-47c9-9c68-4047b59b178b",
    "history_254_69f0add7-09b8-4def-86f8-f196003f0159",
    "history_1773_c2b99dcd-f6a3-4928-ac83-8a88518e7fc1",
    # "nfl_227_ef94ebec-a145-4a1b-b4bc-e817e341fc04",
    "history_1125_781b563a-163a-4029-aa6a-072490987f00",
    "history_2609_139eab9d-3c2f-4d35-b562-b4520bb73c45"
}
# good_questions = {}

# Create an empty DataFrame to store mismatched rows
mismatched_rows = pd.DataFrame(columns=responses.columns)
for idx, row in responses.iterrows():
    response_text = row['answer']
    # if row['id'] in good_questions:
    #     print(row['id'])
    
    # Compare to ground-truth
    gt_row = data.loc[data['id'] == row['id']]
    gt_answer = gt_row['answer'].iloc[0]  # or answer[0], etc.
    if row['id'] in bad_questions:
        continue
    
    if len(gt_answer) > 1 and row['id'] not in good_questions:# skip if there is only one answer
        # print(f"skipped {row['id']}")
        continue
    extracted_answer = extract_answer(response_text)
    # print(f"ID: {row['id']}")
    # print(f"Model Answer Text: {response_text}")
    # print(f"Extracted numeric answer: {extracted_answer}")
    # gt_number = float(gt_answer[-1][0])  # 54.4
    # if extracted_answer != -99999999999999:
    #     continue
    gt1 = str(gt_answer[0][0])
    # gt2 = ""
    # if len(gt_answer) == 2:
    #     gt2 = str(gt_answer[1][0])
    # gt_number = gt1 + " " + gt2
    gt_number = gt1
    row['gt_number'] = float(gt_number)
    # mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)
    # print(f"Ground-truth answer: {gt_number}")
    # print("----")
    
    
    # Check if the extracted answer matches the ground-truth
    # print(extracted_answer, gt_number)
    # if extracted_answer == gt_number:
    if abs(float(extracted_answer) - float(gt_number)) <= 0.00001:
        if extracted_answer == -99999999999999 and row['id'] not in good_questions:
            # print(row['answer'])
            # print(gt_number)
            # print()
            continue
        # if abs(float(extracted_answer) - float(gt_number)) <= 0.001:
        #     continue
        row['gt_number'] = gt_number

        if row['id'] in good_questions:
            print(row['id'])
        mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)
    # if abs(extracted_answer - gt_number) < 0.1: # correct answer
    #     row['gt_number'] = gt_number
    #     mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)
    # if extracted_answer == gt_number: # correct
    #     row['gt_number'] = gt_number
    #     mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)

questionable = {
    "history_2142_373f73fb-83ea-483f-a7c3-1cbed017ad84",
    "history_2060_9368ae36-c505-490c-a08f-0b35cb4a623d",
    "history_2142_d7b01be5-e94c-42b1-b83a-5cadc9e86b38",
    "history_2179_b2940de2-45cc-4727-a5df-bff309816c52",
    "history_1233_30c4acad-427e-41b4-a076-326e0da2e210",
    "history_1863_b73c649b-7d9e-433b-8ac0-c91de4b48a6c",
}
# Save the mismatched rows to a new CSV file
# mismatched_rows.to_csv('drop_correct_responses.csv', index=False)
# mismatched_rows = mismatched_rows.sample(n=45, random_state=42)
# # Drop rows where id is in the questionable set
# mismatched_rows = mismatched_rows[~mismatched_rows['id'].isin(questionable)]
# mismatched_rows.to_csv('DROP_data/correct_responses.csv', index=False)
# mismatched_rows.to_csv('gem_incorrect_drop.csv', index=False)
print(len(mismatched_rows))

720
40


In [44]:
new_csv = '/Users/log/Github/textual_grounding/logan/mismatched_responses.csv'

mismatched_responses = pd.read_csv(new_csv)

print(len(mismatched_responses))
for idx, row in mismatched_responses.iterrows():
    response_text = row['answer']
    
    extracted_answer = extract_answer(response_text)
    # if extracted_answer == -99999999999999.0:
    #     continue
    print(f"ID: {row['id']}")
    # print(f"Model Answer Text: {response_text}")
    print(f"Extracted numeric answer: {extracted_answer}")

    gt_number = row['gt_number']  # 54.4
    print(f"Ground-truth answer: {gt_number}")
    print("----")

95
ID: nfl_1184_7dfd2b64-f39e-4bb4-aeb0-1900adda6018
Extracted numeric answer: 4.0
Ground-truth answer: 2.0
----
ID: nfl_1184_03fdc8a0-d62b-49a0-8087-af92e6659650
Extracted numeric answer: 5.0
Ground-truth answer: 2.0
----
ID: nfl_1332_03d4464c-8509-4a0f-ba98-a495be13a511
Extracted numeric answer: 1.0
Ground-truth answer: 3.0
----
ID: nfl_1332_c87797b8-eefe-4572-b24e-8d8716460746
Extracted numeric answer: 3.0
Ground-truth answer: 2.0
----
ID: history_2170_9b27311d-81ec-4f40-a4af-7ead916d5859
Extracted numeric answer: 9.0
Ground-truth answer: 4.0
----
ID: history_2170_749ed05c-22fc-4ba8-ac83-040be66718d5
Extracted numeric answer: 89.9
Ground-truth answer: 79.9
----
ID: nfl_16_9eb68f5c-0c59-4850-9f2d-e6bbb80cbfa0
Extracted numeric answer: 20.0
Ground-truth answer: 2.0
----
ID: history_1012_31182c98-e949-4810-9a61-f65bfc460210
Extracted numeric answer: 4.0
Ground-truth answer: 3.0
----
ID: history_1959_44b65699-cfb2-4692-93d1-8d771f004430
Extracted numeric answer: 25176.0
Ground-truth ans

In [153]:
csv1 = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/gemini_incorrect_responses.csv')
csv2 = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/llama_incorrect_responses.csv')

# print(len(csv1))
# print(len(csv2))
csv3 = pd.concat([csv1, csv2])
print(len(csv3))

csv3.to_csv('/Users/log/Github/textual_grounding/logan/DROP_data/tagged_combined_incorrect_responses.csv', index=False)

43


In [159]:
df = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/tagged_combined_incorrect_responses.csv')
df_deduplicated = df.drop_duplicates(subset='id', keep='first')

print(len(df_deduplicated))
print(df_deduplicated['id'].value_counts())

33
id
history_2170_77ec7d38-739a-4ca5-9bc2-b63670f435f6    1
history_1863_b73c649b-7d9e-433b-8ac0-c91de4b48a6c    1
history_2609_139eab9d-3c2f-4d35-b562-b4520bb73c45    1
history_1408_7b6e7625-a8ba-4cde-8496-1f6c222b2bc9    1
nfl_1962_20fc3177-88af-4460-b053-dee560576d16        1
nfl_227_ef94ebec-a145-4a1b-b4bc-e817e341fc04         1
nfl_1596_ef82d962-7ffe-481a-b51d-63e60bbf2c2c        1
history_1450_cf455cf1-0221-45c3-88ed-b0c768ed82c5    1
history_241_0ac9c4ec-9ef4-429b-970a-9af43bdb672b     1
history_1418_da95efc1-0a9d-4232-9af5-68dc6e3e69df    1
history_1884_02254a93-0cac-4576-b192-5d46b9b683e9    1
nfl_1390_143a118b-f487-451d-8a10-9a58b9b6b862        1
nfl_2125_92420219-58dd-4c72-87cb-8804cbb50927        1
history_1313_797e963c-52f2-4f8e-ba70-5a382cda0d85    1
history_1954_2c054bf0-1177-4f44-865c-a6e02d20494e    1
history_1853_176835fe-042f-4085-af4a-945ef3f1f903    1
history_2298_7f3056c5-d276-4b3d-933e-0de7ef9e4edd    1
nfl_2262_78f903a9-c9bc-4c21-876a-b955f331cce8        1
nfl_

In [150]:
import pandas as pd

# Load individual CSVs
csv1 = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/tagged_combined_incorrect_responses.csv')
csv2 = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/tagged_correct_responses.csv')
csv3 = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/untagged_combined_incorrect_responses.csv')
csv4 = pd.read_csv('/Users/log/Github/textual_grounding/logan/DROP_data/untagged_correct_responses.csv')

# Define a function to add the required columns and transform the data
def preprocess_data(df, is_true, is_tagged):
    df_preprocessed = pd.DataFrame({
        "id": df["id"],                        # id
        "question": df["answer"],              # question -> from 'answer'
        "gt": df["gt_number"],                        # should be 'gt_number'
        "isTrue": is_true,                     # depends on if correct in file name
        "isTagged": is_tagged,                 # depend on if tag in file name
        "dataset": "DROP",                       # always "DROP"
        "unique_id": df["id"]                  # same as id
    })
    return df_preprocessed

# Process each individual CSV
csv1_processed = preprocess_data(csv1, is_true=0, is_tagged=1)  # Incorrect, Tagged
csv2_processed = preprocess_data(csv2, is_true=1, is_tagged=1)  # Correct, Tagged
csv3_processed = preprocess_data(csv3, is_true=0, is_tagged=0)  # Incorrect, Untagged
csv4_processed = preprocess_data(csv4, is_true=1, is_tagged=0)  # Correct, Untagged

# Merge all processed data into one DataFrame
merged = pd.concat([csv1_processed, csv2_processed, csv3_processed, csv4_processed])

# Save the merged DataFrame to a new CSV
merged.to_csv('/Users/log/Github/textual_grounding/logan/DROP_data/drop_all_responses.csv', index=False)

print(f"Total rows in merged data: {len(merged)}")


Total rows in merged data: 160
